In [ ]:
# 用户对物品的喜欢分类

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

In [2]:
# 数据读取
order_products = pd.read_csv("../data/instacart/order_products__prior.csv")
products = pd.read_csv("../data/instacart/products.csv")
orders = pd.read_csv("../data/instacart/orders.csv")
aisles = pd.read_csv("../data/instacart/aisles.csv")
order_products

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


In [3]:
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [4]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [5]:
aisles

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
...,...,...
129,130,hot cereal pancake mixes
130,131,dry pasta
131,132,beauty
132,133,muscles joints pain relief


In [6]:
# 按索引合并表格
tab1 = pd.merge(aisles, products, on=["aisle_id"])
tab1

,aisle_id,aisle,product_id,product_name,department_id
0,1,prepared soups salads,209,Italian Pasta Salad,20
1,1,prepared soups salads,554,Turkey Chili,20
2,1,prepared soups salads,886,Whole Grain Salad with Roasted Pecans & Mango ...,20
3,1,prepared soups salads,1600,Mediterranean Orzo Salad,20
4,1,prepared soups salads,2539,Original Potato Salad,20
...,...,...,...,...,...
49683,134,specialty wines champagnes,47713,Sparkling Rose,5
49684,134,specialty wines champagnes,48984,Dry Sherry,5
49685,134,specialty wines champagnes,49438,Dry Fino Sherry,5
49686,134,specialty wines champagnes,49562,Blanc De Noirs Sparkling Wine,5


In [7]:
tab2 = pd.merge(tab1, order_products, on=["product_id"])
tab2

,aisle_id,aisle,product_id,product_name,department_id,order_id,add_to_cart_order,reordered
0,1,prepared soups salads,209,Italian Pasta Salad,20,94246,5,0
1,1,prepared soups salads,209,Italian Pasta Salad,20,192465,2,1
2,1,prepared soups salads,209,Italian Pasta Salad,20,195206,18,1
3,1,prepared soups salads,209,Italian Pasta Salad,20,227717,1,1
4,1,prepared soups salads,209,Italian Pasta Salad,20,260072,13,0
...,...,...,...,...,...,...,...,...
32434484,134,specialty wines champagnes,49661,Porto,5,2238789,10,1
32434485,134,specialty wines champagnes,49661,Porto,5,2251344,5,1
32434486,134,specialty wines champagnes,49661,Porto,5,2616249,2,0
32434487,134,specialty wines champagnes,49661,Porto,5,2715237,1,0


In [8]:
tab3 = pd.merge(tab2, orders, on='order_id')
tab3

,aisle_id,aisle,product_id,product_name,department_id,order_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,prepared soups salads,209,Italian Pasta Salad,20,94246,5,0,114082,prior,26,0,20,1.0
1,1,prepared soups salads,22853,Pesto Pasta Salad,20,94246,4,0,114082,prior,26,0,20,1.0
2,4,instant foods,12087,Chicken Flavor Ramen Noodle Soup,9,94246,15,0,114082,prior,26,0,20,1.0
3,4,instant foods,47570,Original Flavor Macaroni & Cheese Dinner,9,94246,14,1,114082,prior,26,0,20,1.0
4,13,prepared meals,10089,Dolmas,20,94246,25,0,114082,prior,26,0,20,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32434484,134,specialty wines champagnes,47713,Sparkling Rose,5,3014872,1,0,63218,prior,1,1,14,NaN
32434485,134,specialty wines champagnes,49562,Blanc De Noirs Sparkling Wine,5,34570,1,1,37901,prior,13,3,9,13.0
32434486,134,specialty wines champagnes,49562,Blanc De Noirs Sparkling Wine,5,250923,1,1,26431,prior,27,5,11,10.0
32434487,134,specialty wines champagnes,49562,Blanc De Noirs Sparkling Wine,5,1319402,1,1,26431,prior,34,2,14,7.0


In [9]:
# 交叉表
table = pd.crosstab(tab3["user_id"], tab3["aisle"])
table

aisle,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,0,3,0,0,0,0,2,0,0,0,...,3,1,1,0,0,0,0,2,0,42
3,0,0,0,0,0,0,0,0,0,0,...,4,1,0,0,0,0,0,2,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
5,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206205,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
206206,0,4,0,0,0,0,4,1,0,0,...,1,0,0,0,0,1,0,1,0,0
206207,0,0,0,0,1,0,0,0,0,0,...,3,4,0,2,1,0,0,11,0,15


In [10]:
# 取前10000个样本
data = table[:10000]
data

aisle,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,0,3,0,0,0,0,2,0,0,0,...,3,1,1,0,0,0,0,2,0,42
3,0,0,0,0,0,0,0,0,0,0,...,4,1,0,0,0,0,0,2,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
5,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,3,0,0,0,0,0,1,0,0,0,...,2,0,0,1,0,0,0,0,2,1
9997,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,8
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# PCA降维
transfer = PCA(n_components=0.95)
data_new = transfer.fit_transform(data)
data_new

array([[-2.36456828e+01,  2.30028678e+00, -2.71706275e+00, ...,
         8.24685231e-01, -5.20365905e-01,  2.99847178e+00],
       [ 6.95477119e+00,  3.54966052e+01,  2.52655545e+00, ...,
        -1.15326520e+00, -1.37969318e+00, -1.07115466e-02],
       [-7.47792843e+00,  2.83147785e+00, -1.07306519e+01, ...,
        -3.55104796e-01,  3.85595697e-02,  1.92318882e+00],
       ...,
       [-2.86664024e+01, -1.26446961e+00, -1.14908062e+00, ...,
         1.13859569e-03,  4.14189764e-01,  4.46163585e-01],
       [-2.88378748e+01, -1.70490822e+00, -1.47059942e+00, ...,
        -1.62743887e-01,  6.72795951e-01,  9.64403654e-02],
       [ 2.10412407e+02, -3.51935647e+01,  1.33671987e+01, ...,
         1.46544596e+01,  1.56764794e+01,  1.67432890e+01]])

In [12]:
data_new.shape

(10000, 42)

In [13]:
# 模型训练
estimator = KMeans(n_clusters=3)
estimator.fit(data_new)

D:\Soft\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=3)

In [14]:
# 模型预测
y_predict = estimator.predict(data_new)
y_predict[:300]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1,
       0, 1, 2, 1, 1, 1, 0, 1, 1, 1, 1, 2, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 2, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0])

In [15]:
# 模型评估-轮廓系数SC
silhouette_score(data_new, y_predict)

0.540501956267417

### 通过KMeans后的结果进行分类

In [16]:
# 数据集添加KMeans聚类后的标签
data["Class"] = y_predict
data

C:\Users\Administrator\AppData\Local\Temp\ipykernel_11936\556197507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Class"] = y_predict


aisle,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt,Class
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,0,3,0,0,0,0,2,0,0,0,...,1,1,0,0,0,0,2,0,42,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,2,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
5,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,3,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,2,1,1
9997,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,8,1
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
